In [11]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2

In [12]:
df_train = pd.read_csv('imdb_sup.csv')

In [13]:
df_train.head()

,Review,Rating,Sentiment
0,"Kurt Russell's chameleon-like performance, coupled with John Carpenter's flawless filmmaking, makes this one, without a doubt, one of the finest boob-tube bios ever aired. It holds up, too: the emotional foundation is strong enough that it'll never age; Carpenter has preserved for posterity the power and ultimate poignancy of the life of the one and only King of Rock and Roll. (I'd been a borderline Elvis fan most of my life, but it wasn't until I saw this mind-blowingly moving movie that I looked BEYOND the image at the man himself. It was quite a revelation.) ELVIS remains one of the top ten made-for-tv movies of all time.",10,1
1,"It was extremely low budget(it some scenes it looks like they recorded with a home video recorder). However it does have a good plot line, and its easy to follow. 8 years after shooting her sexually abusive step father Amanda is released from the psychiatric ward, with the help of her doctor who she is secretly having an affair with. The doctor ends up renting her a house and buying her a car. But within the first 20 minutes of the movie Amanda kills him and buries him in her backyard. Then she see's her neighbor Richard sets eyes on him and stops at nothing until she has him. She acts innocent but after another neighbor Buzz finds out that Amanda killed that doctor and attempted to kill Richards wife Laurie (this is after Amanda and him get it on in the hot tub). Then she stops acting so Innocent and kills Buzz and later on attempts to kill Richard whom she supposedly loves and cares for. And you'll have to rent the movie to find out if Amanda dies or not. Overall good movie, reminds me a lot of my life you know the whole falling for the neighbor and stopping at nothing until you have him part.",8,1
2,"James Cagney is best known for his tough characters- and gangster roles but he has also played quite a lot 'soft' characters in his career. This musical is one of them and it was the first but not the last musical movie Cagney would star in.<br /><br />Cagney is even doing a bit of singing in this one and also quite an amount of dancing. And it needs to be said that he was not bad at it. He plays the role with a lot of confidence. He apparently had some dancing jobs in his early life before his acting career started to take off big time, so it actually isn't a weird thing that he also took on some musical acting roles in his career. He obviously also feels at ease in this totally different genre than most people are accustomed to seeing him in.<br /><br />The movie is directed by Lloyd Bacon, who was perhaps among the best and most successful director within the genre. His earliest '30's musicals pretty much defined the musical genre and he also was responsible for genre movies such as ""42nd Street"". His musicals were always light and fun to watch and more comedy like than anything else really. '30's musicals never were really about its singing, this was something that more featured in '40's and later made musicals, mainly from the MGM studios.<br /><br />As usual it has a light and simple story, set in the musical world, that of course is also predictable and progresses in a formulaic way. It nevertheless is a fun and simple story that also simply makes this an entertaining movies to watch. So do the characters and actors that are portraying them. Sort of weird though that that the total plot line of the movie gets sort of abandoned toward the end of the movie, when the movie only starts to consists out of musical number routines.<br /><br />The musical moments toward the ending of the movie are also amusing and well done, even though I'm not a too big fan of the genre itself. Once again the musical numbers also feature a young Billy Barty. he often played little boys/babies/mice and whatever more early on in his career, including the movie musical ""Gold Diggers of 1933"", of one year earlier. <br /><br />A recommendable early genre movie.<br /><br />8/10",8,1
3,"Fol

In [14]:
df_train['Rating'].unique()

array([10,  8,  7,  9,  3,  2,  1,  4])

In [15]:
# Define the mapping function
def map_sentiment(rating):
    if rating in [1, 2, 3]:
        return 0  # Negative
    elif rating in [4, 7]:
        return 1  # Neutral
    elif rating in [8, 9, 10]:
        return 2  # Positive
    else:
        return None  # Handle unexpected values

# Apply the function to the Sentiment column
df_train["Sentiment"] = df_train["Rating"].apply(map_sentiment)

# Check the updated Sentiment column
df_train["Sentiment"].value_counts()


Sentiment
2    20197
0    19669
1    10134
Name: count, dtype: int64

In [16]:
df_train.drop(columns=["Rating"], inplace=True)

In [17]:
df_train.head()

,Review,Sentiment
0,"Kurt Russell's chameleon-like performance, coupled with John Carpenter's flawless filmmaking, makes this one, without a doubt, one of the finest boob-tube bios ever aired. It holds up, too: the emotional foundation is strong enough that it'll never age; Carpenter has preserved for posterity the power and ultimate poignancy of the life of the one and only King of Rock and Roll. (I'd been a borderline Elvis fan most of my life, but it wasn't until I saw this mind-blowingly moving movie that I looked BEYOND the image at the man himself. It was quite a revelation.) ELVIS remains one of the top ten made-for-tv movies of all time.",2
1,"It was extremely low budget(it some scenes it looks like they recorded with a home video recorder). However it does have a good plot line, and its easy to follow. 8 years after shooting her sexually abusive step father Amanda is released from the psychiatric ward, with the help of her doctor who she is secretly having an affair with. The doctor ends up renting her a house and buying her a car. But within the first 20 minutes of the movie Amanda kills him and buries him in her backyard. Then she see's her neighbor Richard sets eyes on him and stops at nothing until she has him. She acts innocent but after another neighbor Buzz finds out that Amanda killed that doctor and attempted to kill Richards wife Laurie (this is after Amanda and him get it on in the hot tub). Then she stops acting so Innocent and kills Buzz and later on attempts to kill Richard whom she supposedly loves and cares for. And you'll have to rent the movie to find out if Amanda dies or not. Overall good movie, reminds me a lot of my life you know the whole falling for the neighbor and stopping at nothing until you have him part.",2
2,"James Cagney is best known for his tough characters- and gangster roles but he has also played quite a lot 'soft' characters in his career. This musical is one of them and it was the first but not the last musical movie Cagney would star in.<br /><br />Cagney is even doing a bit of singing in this one and also quite an amount of dancing. And it needs to be said that he was not bad at it. He plays the role with a lot of confidence. He apparently had some dancing jobs in his early life before his acting career started to take off big time, so it actually isn't a weird thing that he also took on some musical acting roles in his career. He obviously also feels at ease in this totally different genre than most people are accustomed to seeing him in.<br /><br />The movie is directed by Lloyd Bacon, who was perhaps among the best and most successful director within the genre. His earliest '30's musicals pretty much defined the musical genre and he also was responsible for genre movies such as ""42nd Street"". His musicals were always light and fun to watch and more comedy like than anything else really. '30's musicals never were really about its singing, this was something that more featured in '40's and later made musicals, mainly from the MGM studios.<br /><br />As usual it has a light and simple story, set in the musical world, that of course is also predictable and progresses in a formulaic way. It nevertheless is a fun and simple story that also simply makes this an entertaining movies to watch. So do the characters and actors that are portraying them. Sort of weird though that that the total plot line of the movie gets sort of abandoned toward the end of the movie, when the movie only starts to consists out of musical number routines.<br /><br />The musical moments toward the ending of the movie are also amusing and well done, even though I'm not a too big fan of the genre itself. Once again the musical numbers also feature a young Billy Barty. he often played little boys/babies/mice and whatever more early on in his career, including the movie musical ""Gold Diggers of 1933"", of one year earlier. <br /><br />A recommendable early genre movie.<br /><br />8/10",2
3,"Following the bri

In [18]:
df_train["Sentiment"].value_counts()


Sentiment
2    20197
0    19669
1    10134
Name: count, dtype: int64

In [19]:
# Set Pandas to display full text
pd.set_option("display.max_colwidth", None)

# Get 5 neutral reviews
neutral_reviews = df_train[df_train["Sentiment"] == 1].head(5)

# Display full reviews
print(neutral_reviews)


In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle  # To save and load tokenizer and label encoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Load dataset
df = df_train  # Ensure dataset is loaded

# Step 1: Data Cleaning (Ensure text is cleaned)
df["Review"] = df["Review"].astype(str).str.lower()

# Step 2: Convert Sentiments to Categories (One-Hot Encoding)
label_encoder = LabelEncoder()
df["Sentiment"] = label_encoder.fit_transform(df["Sentiment"])  # 0, 1, 2

# Step 3: Tokenization and Padding
MAX_WORDS = 5000  # Vocabulary size
MAX_LEN = 200  # Max length of review

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(df["Review"])

X = tokenizer.texts_to_sequences(df["Review"])
X = pad_sequences(X, maxlen=MAX_LEN)

y = to_categorical(df["Sentiment"], num_classes=3)  # Convert labels to categorical

# Step 4: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Build LSTM Model
model = Sequential([
    Embedding(MAX_WORDS, 128, input_length=MAX_LEN),
    LSTM(128, return_sequences=True),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(3, activation="softmax")  # 3 classes (Negative, Neutral, Positive)
])

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Step 6: Train the Model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Step 7: Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# ---- SAVE MODEL, TOKENIZER & LABEL ENCODER ----
model.save("sentiment_model.h5")
with open("tokenizer.pkl", "wb") as handle:
    pickle.dump(tokenizer, handle)
with open("label_encoder.pkl", "wb") as handle:
    pickle.dump(label_encoder, handle)
print("Model, tokenizer, and label encoder saved successfully!")

# ---- LOAD MODEL, TOKENIZER & LABEL ENCODER ----
model = load_model("sentiment_model.h5")
with open("tokenizer.pkl", "rb") as handle:
    tokenizer = pickle.load(handle)
with open("label_encoder.pkl", "rb") as handle:
    label_encoder = pickle.load(handle)
print("Model, tokenizer, and label encoder loaded successfully!")

# ---- FUNCTION TO TAKE USER INPUT & PREDICT ----
def predict_sentiment(user_input):
    user_input = user_input.lower()
    sequence = tokenizer.texts_to_sequences([user_input])
    padded_sequence = pad_sequences(sequence, maxlen=MAX_LEN)
    prediction = model.predict(padded_sequence)
    predicted_label = np.argmax(prediction)
    sentiment = label_encoder.inverse_transform([predicted_label])
    return sentiment[0]

# ---- USER INPUT LOOP ----
while True:
    user_review = input("\nEnter a movie review (or type 'exit' to quit): ")
    if user_review.lower() == "exit":
        break
    sentiment = predict_sentiment(user_review)
    print(f"Predicted Sentiment: {sentiment}")


C:\Users\BIT\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 248s 196ms/step - accuracy: 0.6038 - loss: 0.8601 - val_accuracy: 0.6768 - val_loss: 0.7579
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 255s 191ms/step - accuracy: 0.7082 - loss: 0.7077 - val_accuracy: 0.7370 - val_loss: 0.6569
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 246s 196ms/step - accuracy: 0.7611 - loss: 0.6007 - val_accuracy: 0.7170 - val_loss: 0.7159
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 250s 200ms/step - accuracy: 0.7712 - loss: 0.5684 - val_accuracy: 0.7320 - val_loss: 0.6474
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 252s 202ms/step - accuracy: 0.8064 - loss: 0.4925 - val_accuracy: 0.7496 - val_loss: 0.6437
313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 75ms/step - accuracy: 0.7450 - loss: 0.6539


Test Accuracy: 0.7496


Model, tokenizer, and label encoder saved successfully!
Model, tokenizer, and label encoder loaded successfully!



Enter a movie review (or type 'exit' to quit):  harry potter was a bad movie. I hated the ending


...


Enter a movie review (or type 'exit' to quit):  it was a good movie


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Predicted Sentiment: 1



Enter a movie review (or type 'exit' to quit):  It was a very good movie i liked it


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Predicted Sentiment: 2



Enter a movie review (or type 'exit' to quit):  exit
